<a href="https://colab.research.google.com/github/nielseni6/ADV-EMRG-TPCS-IN-CI-ML-AND-DM/blob/master/Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''

Q3 Inception Module for CIFAR-10 dataset

'''
import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, MaxPool2D
from keras.layers import Flatten, Dense, GlobalAvgPool2D
from keras.layers import Input
from keras.utils import np_utils
from keras.datasets import cifar10

def inception_block(inp_img):
  # Create Volumes for the Inception module
  volume_1 = Conv2D(64, (1,1), padding='same', activation='relu')(inp_img)

  volume_2 = Conv2D(96, (1,1), padding='same', activation='relu')(inp_img)
  volume_2 = Conv2D(128, (3,3), padding='same', activation='relu')(volume_2)

  volume_3 = Conv2D(16, (1,1), padding='same', activation='relu')(inp_img)
  volume_3 = Conv2D(32, (5,5), padding='same', activation='relu')(volume_3)

  volume_4 = MaxPooling2D((3,3), strides=(1,1), padding='same')(inp_img)
  volume_4 = Conv2D(32, (1,1), padding='same', activation='relu')(volume_4)

  # Concatenate all volumes of the Inception module
  return keras.layers.concatenate([volume_1, volume_2, volume_3,
                                              volume_4], axis = 3)

epochs = 100

# Get the data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Get the data ready
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# Create imput
input_img = Input(shape = (32, 32, 3))

#layer1 = Conv2D(64, (7,7), strides=2, padding='same', activation='relu')(input_img)
layer1 = inception_block(input_img)
layer2 = MaxPool2D(pool_size=(3,3), strides=2, padding='same')(layer1)

inception_module1 = inception_block(layer2)
inception_module1 = MaxPool2D(pool_size=(3,3), strides=2, padding='same')(inception_module1)

inception_module2 = inception_block(inception_module1)
inception_module = GlobalAvgPool2D()(inception_module2)

output = Flatten()(inception_module)
out    = Dense(10, activation='softmax')(output)


model = Model(inputs = input_img, outputs = out)
print(model.summary())

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
hist = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=512)


scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))




Model: "functional_33"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_25 (InputLayer)           [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_662 (Conv2D)             (None, 32, 32, 96)   384         input_25[0][0]                   
__________________________________________________________________________________________________
conv2d_664 (Conv2D)             (None, 32, 32, 16)   64          input_25[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_158 (MaxPooling2D (None, 32, 32, 3)    0           input_25[0][0]                   
______________________________________________________________________________________